# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

2024-02-14 12:25:15.080388: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 12:25:15.121036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-14 12:25:16.604762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 데이터 셋 읽어오기

In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

타겟 변수 'is_converted' 데이터 확인

In [5]:
df_train['is_converted'].value_counts() # 데이터 불균형이 심하다. 

is_converted
False    54449
True      4850
Name: count, dtype: int64

데이터 결측치 확인

In [6]:
df_train.isnull().sum() # 결측치가 심하다. 제거 전에 상관관계를 확인해보자. 

bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64

In [7]:
def corr_with_target(df,variable1,variable2):
    # 두 변수 간의 상관 계수를 계산합니다.
    correlation = df[variable1].corr(df[variable2])

    # 상관 계수를 출력합니다.
    print(f"상관 계수 between {variable1} and {variable2}: {correlation}")

corr_with_target(df_train,'historical_existing_cnt','is_converted')

상관 계수 between historical_existing_cnt and is_converted: -0.004489584471512195


In [ ]:
'''
# string -> int로 변환 후 실행 해야 함
for column_name in df_train.columns:
    if df_train[column_name].isnull().sum() >= 50000:
        print(column_name)
        print(corr_with_target(df_train, column_name, 'is_converted'))
    else:
        pass
'''

In [ ]:
# False 값을 가진 행의 인덱스를 추출.
false_indices = df_train[df_train['customer_country.1'] != df_train['customer_country']].index

# False 값을 가진 행의 인덱스와 해당 값들을 출력.
for idx in false_indices:
    print("인덱스:", idx)
    print("customer_country.1 값:", df_train.at[idx, 'customer_country.1'])
    print("customer_country 값:", df_train.at[idx, 'customer_country'])
    print()

In [ ]:
df_train['response_corporate'].value_counts()

## 2. 데이터 전처리

In [10]:
df_train.columns

Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted'],
      dtype='object')

### 'customer_type' value 값 확인
-> End-Customer, End Customer => 통합해야 함    
-> Other, Others, Etcs.

In [9]:
df_train['customer_type'].value_counts(dropna = False)

customer_type
NaN                             43961
End-Customer                     6647
End Customer                     3996
Specifier/ Influencer            2525
Channel Partner                  1368
Service Partner                   349
Solution Eco-Partner              146
Installer/Contractor               52
Specifier / Influencer             43
Corporate                          31
HVAC Engineer                      23
Engineer                           20
Developer                          17
Technician                         16
Consultant                         15
Other                              10
Home Owner                         10
End-user                            8
Manager / Director                  8
Software/Solution Provider          7
Etc.                                6
Architect/Consultant                5
Homeowner                           5
Reseller                            5
Installer                           5
Interior Designer                   

In [11]:
df_train['ver_cus'].value_counts(dropna = False)

ver_cus
0    56832
1     2467
Name: count, dtype: int64

In [20]:
df_train[df_train['ver_cus'] == 1]['customer_type'].unique()

array(['End-Customer', 'End Customer', 'End-user'], dtype=object)

In [21]:
df_train[df_train['ver_cus'] == 0]['customer_type'].unique()

array(['Specifier/ Influencer', 'Service Partner', 'Channel Partner', nan,
       'Corporate', 'End-Customer', 'Specifier / Influencer',
       'Solution Eco-Partner', 'Distributor', 'Other', 'End Customer',
       'System Integrator', 'Consultant', 'Installer', 'Homeowner',
       'Others', 'Technician', 'Installer/Contractor', 'Engineer',
       'Manager / Director', 'Developer', 'Etc.', 'Architect/Consultant',
       'HVAC Engineer', 'Reseller', 'Software/Solution Provider',
       'End-user', 'Software / Solution Provider', 'Dealer/Distributor',
       'Technical Assistant', 'Commercial end-user', 'Interior Designer',
       'Home Owner', 'Administrator'], dtype=object)

In [28]:
df_train[(df_train['ver_cus'] == 0) & (df_train['customer_type'] == 'Etc.')]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
19051,0.75,/MEXICALI/Mexico,AS,NaN,43565,Etc.,Enterprise,NaN,NaN,NaN,...,LGEMS,NaN,0,0,NaN,NaN,NaN,NaN,42,False
19052,1.00,/MEXICALI/Mexico,AS,NaN,43565,Etc.,Enterprise,NaN,NaN,NaN,...,LGEMS,more than a year,0,0,NaN,NaN,NaN,NaN,42,False
29343,0.75,Carrera 29 # 45-45 piso 15 8902130749 /Bucara...,AS,0.003937,3134,Etc.,SMB,0.0,NaN,NaN,...,LGECB,NaN,0,0,0.003079,0.026846,corporate / office,NaN,648,False
36453,0.75,/Bogota/Colombia,AS,NaN,43366,Etc.,SMB,NaN,NaN,NaN,...,LGECB,NaN,0,0,NaN,NaN,NaN,NaN,39,False
45088,1.00,via G. Pascoli 39 /Milano/Italy,AS,NaN,16210,Etc.,Enterprise,NaN,NaN,NaN,...,LGEIS,3 months ~ 6 months,0,0,NaN,NaN,NaN,NaN,341,False
55208,1.00,/cali/Colombia,AS,0.003788,1589,Etc.,Enterprise,0.0,NaN,NaN,...,LGECB,3 months ~ 6 months,0,0,0.000298,0.020121,residential (home),NaN,101,False


In [45]:
df_train[(df_train['ver_cus'] == 1) & (df_train['customer_type'] == 'End Customer')]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
318,1.00,/Rio Grande do Sul/Brazil,AS,0.003937,27309,End Customer,SMB,NaN,NaN,NaN,...,LGESP,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,IT/Software,97,False
321,1.00,/East London/South Africa,AS,0.040816,30958,End Customer,SMB,NaN,NaN,NaN,...,LGESA,more than a year,1,0,0.003079,0.026846,corporate / office,IT/Software,26,False
322,1.00,/East London/South Africa,AS,0.040816,30958,End Customer,SMB,NaN,NaN,NaN,...,LGESA,more than a year,1,0,0.003079,0.026846,corporate / office,IT/Software,26,False
323,1.00,/Baghdad/Iraq,AS,0.040816,41452,End Customer,SMB,NaN,NaN,NaN,...,LGELF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,117,False
327,1.00,"/NCR, Quezon City/Philippines",AS,0.066667,6634,End Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,NaN,119,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58168,0.25,7665 Corporate Center Dr. Cruise/Miami/United...,ID,NaN,30454,End Customer,SMB,NaN,1.0,NaN,...,LGEUS,NaN,1,0,0.000717,0.071345,hotel & accommodation,Cruise,570,False
58169,0.25,7665 Corporate Center Dr. Cruise/Miami/United...,ID,NaN,30455,End Customer,SMB,NaN,1.0,NaN,...,LGEUS,NaN,1,0,0.000717,0.071345,hotel & accommodation,Cruise,570,True
58170,0.25,903 44th Rd Cruise/Long Island City/United St...,ID,NaN,28954,End Customer,SMB,NaN,1.0,NaN,...,LGEUS,NaN,1,0,0.000717,0.071345,hotel & accommodation,Cruise,570,True
58171,0.25,3655 NW 87th Ave Commercial Display/Miami/Uni...,ID,NaN,7741,End Customer,SMB,0.0,1.0,NaN,...,LGEUS,NaN,1,0,0.000717,0.071345,hotel & accommodation,Cruise,570,True


In [46]:
df_train[(df_train['ver_cus'] == 1) & (df_train['customer_type'] == 'End Customer')]['business_area'].value_counts(dropna = False)

business_area
corporate / office       298
retail                   255
education                135
hotel & accommodation    110
Name: count, dtype: int64

In [47]:
df_train[(df_train['ver_cus'] == 0) & (df_train['customer_type'] == 'End Customer')]['business_area'].value_counts(dropna = False)

business_area
NaN                               1972
hospital & health care             763
special purpose                    148
residential (home)                  88
factory                             84
government department               47
public facility                     40
power plant / renewable energy      29
transportation                      27
Name: count, dtype: int64

In [48]:
df_train[(df_train['ver_cus'] == 1) & (df_train['customer_type'] == 'End-Customer')]['business_area'].value_counts(dropna = False)

business_area
corporate / office       815
retail                   509
education                334
hotel & accommodation      7
Name: count, dtype: int64

In [49]:
df_train[(df_train['ver_cus'] == 0) & (df_train['customer_type'] == 'End-Customer')]['business_area'].value_counts(dropna = False)

business_area
NaN                               3657
special purpose                    329
residential (home)                 263
hospital & health care             231
factory                            217
government department              110
public facility                     89
transportation                      47
power plant / renewable energy      39
Name: count, dtype: int64

In [51]:
df_train[(df_train['ver_cus'] == 1) & (df_train['customer_type'] == 'End-user')]['business_area'].value_counts(dropna = False)

business_area
education    3
retail       1
Name: count, dtype: int64

In [44]:
df_train[df_train['ver_cus'] == 1]['business_area'].value_counts(dropna = False)

business_area
corporate / office       1113
retail                    765
education                 472
hotel & accommodation     117
Name: count, dtype: int64

df_train['ver_cus']의 특정 Vertical Level 1(사업영역)을 고객의 사업영역('business_area')에서 유추한 결과, 4가지 영역에 해당됨   
- corporate / office
- retail -> End-customer
- education -> End-user
- hotel & accommodation


* End-customer은 직접 구매한 사람
* End-user는 실제로 사용하는 사람

In [ ]:
## End-Customer 과 End Customer 통합하기 


### 데이터 결측치 처리

1. df_train['id_strategic_ver']의 결측치 채우기  -> null값 다 0으로 처리하기

In [34]:
df_train['business_unit'].value_counts(dropna = False)

business_unit
ID          25563
AS          24774
IT           8664
Solution      296
CM              2
Name: count, dtype: int64

In [35]:
df_train['id_strategic_ver'].value_counts(dropna = False)

id_strategic_ver
NaN    55855
1.0     3444
Name: count, dtype: int64

In [36]:
df_train[df_train['id_strategic_ver'] == 1.0]['business_unit'].unique()

array(['ID'], dtype=object)

In [43]:
df_train[df_train['id_strategic_ver'] == 1.0]['business_area'].value_counts(dropna = False)

business_area
corporate / office       2460
hotel & accommodation     984
Name: count, dtype: int64

In [73]:
df_train[df_train['id_strategic_ver'] == 1.0]['business_unit'].value_counts(dropna = False)

business_unit
ID    3444
Name: count, dtype: int64

In [74]:
df_train[df_train['id_strategic_ver'] != 1.0]['business_unit'].value_counts(dropna = False)

business_unit
AS          24774
ID          22119
IT           8664
Solution      296
CM              2
Name: count, dtype: int64

In [76]:
df_train[df_train['id_strategic_ver'] != 1.0]['business_area'].value_counts(dropna = False)
# 여기서 왜 cor과 hotel이 나와 했지만.... 여기서 나온 이유는 business_unit이 ID가 아니라서!! 나온거임 

business_area
NaN                               40882
retail                             3011
special purpose                    1929
residential (home)                 1818
education                          1727
corporate / office                 1637
hospital & health care             1199
factory                            1035
hotel & accommodation              1029
government department               650
public facility                     519
transportation                      306
power plant / renewable energy      113
Name: count, dtype: int64

In [75]:
df_train[(df_train['id_strategic_ver'] != 1.0) & (df_train['business_unit'] == 'ID')]['business_area'].value_counts(dropna = False)

business_area
NaN                               15636
retail                             2028
education                          1356
special purpose                    1075
factory                             540
government department               421
residential (home)                  385
public facility                     271
transportation                      216
hospital & health care              148
power plant / renewable energy       43
Name: count, dtype: int64

df_train['id_strategic_ver']의 특정 사업 영역(Vertical Level1)은 고객의 사업 영역을 통해 유추한 결과,   
- corporate / office
- hotel & accommodation

In [78]:
df_train['id_strategic_ver'].isna().sum()
# df_train['id_strategic_ver'] 결측치를 다 0으로 채우기 
# df_test[id_strategic_ver]의 결측치도 다 0으로 채우기


# 코드 작성! 

55855

2. df_train['it_strategic_ver']의 결측치 채우기 -> null값 0으로 대체

In [72]:
df_train['business_unit'].value_counts(dropna = False)

business_unit
ID          25563
AS          24774
IT           8664
Solution      296
CM              2
Name: count, dtype: int64

In [79]:
df_train['it_strategic_ver'].value_counts(dropna = False)

it_strategic_ver
NaN    58178
1.0     1121
Name: count, dtype: int64

In [80]:
df_train[df_train['it_strategic_ver'] == 1.0]['business_unit'].unique()

array(['IT'], dtype=object)

In [81]:
df_train[df_train['it_strategic_ver'] == 1.0]['business_area'].value_counts(dropna = False)

business_area
hotel & accommodation    791
corporate / office       330
Name: count, dtype: int64

In [82]:
df_train[(df_train['it_strategic_ver'] != 1.0) & (df_train['business_unit'] == 'IT')]['business_area'].value_counts(dropna = False)

business_area
NaN                               5972
hospital & health care             895
retail                             163
education                          150
special purpose                    108
factory                             83
government department               49
residential (home)                  42
public facility                     40
transportation                      33
power plant / renewable energy       8
Name: count, dtype: int64

df_train['it_strategic_ver']의 특정 사업 영역(Vertical Level1)은 고객의 사업 영역을 통해 유추한 결과,   
- corporate / office
- hotel & accommodation

In [84]:
df_train['it_strategic_ver'].isna().sum()
# df_train['it_strategic_ver'] 결측치를 다 0으로 채우기 
# df_test[it_strategic_ver]의 결측치도 다 0으로 채우기


# 코드 작성! 

58178

In [88]:
df_train['idit_strategic_ver'].value_counts(dropna = False)

idit_strategic_ver
NaN    54734
1.0     4565
Name: count, dtype: int64

3. df_train['idit_strategic_ver']의 결측치 채우기 -> null값 0으로 대체

In [91]:
df_train[(df_train['id_strategic_ver'] == 1.0) | (df_train['it_strategic_ver'] == 1.0)]['idit_strategic_ver'].value_counts(dropna = False)

idit_strategic_ver
1.0    4565
Name: count, dtype: int64

In [92]:
df_train['idit_strategic_ver'].isna().sum()
# df_train['idit_strategic_ver'] 결측치를 다 0으로 채우기 
# df_test[idit_strategic_ver]의 결측치도 다 0으로 채우기


# 코드 작성!

54734

4. customer_country 변수 처리 -> train과 test 모두에서 drop하기 

In [96]:
df_train['customer_country'].isna().sum()

982

In [94]:
df_train['customer_country'].value_counts(dropna = False)

customer_country
//India                                   3037
/São Paulo/Brazil                         1344
NaN                                        982
//United States                            962
//United Kingdom                           805
                                          ... 
/Hydrated/India                              1
/Tirurangadi, Malappuram, Kerala/India       1
/Canoinhas / SC/Brazil                       1
/Tupã/SP/Brazil                              1
/paris/France                                1
Name: count, Length: 15400, dtype: int64

In [101]:
df_train[df_train['response_corporate'] == 'LGEIL']['customer_country'].unique()

array(['/Kolkata /India', '/Bhubaneswar/India', '/Hyderabad/India', ...,
       '/Rajkot, Gujarat, India/India',
       '7700 Gateway Blvd  Commercial Display/Newark/India',
       '/burla parit/India'], dtype=object)

In [114]:
df_train[df_train['response_corporate'] == 'LGEUK']['customer_country'].unique()

array(['/Bedford/United Kingdom', '//United Kingdom',
       '/Kent/United Kingdom', '/Bangladesh/United Kingdom',
       '/London/United Kingdom', '/Brighton/United Kingdom',
       '/Greater Manchester/United Kingdom', '//Ireland',
       '/Knutsford/United Kingdom', '/Nottinghamshire/United Kingdom',
       '/Dublin/Ireland', '/County/State/United Kingdom',
       '/united kingdom/United Kingdom', '/INDIA/United Kingdom',
       '/Leicestershire/United Kingdom', '/United Kingdom/United Kingdom',
       nan, '/Uk/United Kingdom', '/Leicesterhsire/United Kingdom',
       '/Dublin 2/United Kingdom', '/Oxfordshire/United Kingdom',
       '/Hampshire/United Kingdom', '/Essex/United Kingdom',
       '/United Kingdom', '/Seoul/United Kingdom',
       '/London /United Kingdom', '/england/United Kingdom',
       '/West Yorkshire/United Kingdom', '/dublin/Ireland',
       '/UK/United Kingdom', '/England/United Kingdom',
       '/london/United Kingdom', '/Surrey/United Kingdom',
       '/North

In [128]:
df_train[df_train['customer_country'] == '//']

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
398,0.75,//,ID,0.084337,44601,Channel Partner,SMB,NaN,1.0,NaN,...,LGEIN,3 months ~ 6 months,0,1,0.003079,0.064566,corporate / office,Others,154,True
670,1.00,//,ID,0.074949,17993,NaN,SMB,NaN,1.0,NaN,...,LGECB,3 months ~ 6 months,0,0,0.003079,0.064566,corporate / office,NaN,212,False
671,1.00,//,ID,0.074949,21216,NaN,SMB,NaN,1.0,NaN,...,LGECB,3 months ~ 6 months,0,0,0.003079,0.064566,corporate / office,NaN,230,False
672,1.00,//,ID,0.074949,35054,NaN,SMB,NaN,1.0,NaN,...,LGECB,3 months ~ 6 months,0,0,0.003079,0.064566,corporate / office,NaN,187,False
673,1.00,//,ID,0.074949,19465,NaN,Enterprise,NaN,1.0,NaN,...,LGECB,3 months ~ 6 months,0,0,0.003079,0.064566,corporate / office,NaN,231,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55986,1.00,//,ID,0.054422,37734,NaN,Enterprise,0.0,NaN,NaN,...,LGECB,3 months ~ 6 months,0,0,0.000097,0.079412,government department,NaN,233,False
55987,1.00,//,ID,0.054422,17892,NaN,SMB,NaN,NaN,NaN,...,LGECB,3 months ~ 6 months,0,0,0.000097,0.079412,government department,NaN,233,False
56228,0.50,//,IT,NaN,38852,NaN,Enterprise,NaN,NaN,NaN,...,LGEPS,NaN,0,0,0.000097,NaN,government department,NaN,875,False
56231,0.25,//,IT,NaN,38852,NaN,Enterprise,NaN,NaN,NaN,...,LGEPS,NaN,0,0,0.000097,NaN,government department,NaN,875,False


In [113]:
df_train[df_train['customer_country.1'].isnull()]['response_corporate'].value_counts()

response_corporate
LGESP    558
LGEUK    123
LGEDG     97
LGEGF     45
LGEIL     26
LGEFS     13
LGESJ     10
LGEES     10
LGEPH      9
LGEVH      8
LGEPS      8
LGEAR      8
LGEIN      7
LGEMS      6
LGEKR      6
LGECL      5
LGESA      5
LGEAF      4
LGETK      4
LGEEG      4
LGEML      3
LGESL      3
LGEPR      3
LGEHK      2
LGEPL      2
LGEIS      2
LGECI      2
LGEEF      2
LGEHS      1
LGELF      1
LGETH      1
LGEJP      1
LGEBN      1
LGESW      1
LGEAP      1
Name: count, dtype: int64

In [95]:
df_train['response_corporate'].value_counts(dropna = False)

response_corporate
LGEIL    16908
LGESP     9311
LGEUS     5955
LGEMS     2768
LGEPH     2651
LGEGF     2149
LGECB     2079
LGEUK     1651
LGESJ     1469
LGECL     1339
LGEPS     1252
LGEIS     1146
LGEPR     1131
LGEDG     1051
LGEPL      850
LGEEG      704
LGEVH      614
LGEES      592
LGETK      541
LGEAR      491
LGEKR      433
LGEHK      383
LGEAP      366
LGESL      348
LGEMK      331
LGEFS      302
LGEAF      288
LGEIN      281
LGELF      274
LGESA      268
LGECI      263
LGETH      185
LGEEF      159
LGEPT      111
LGEML      110
LGEBN       99
LGEYK       78
LGECH       65
LGEHS       51
LGETT       47
LGEJP       41
LGEAS       33
LGESW       31
LGEMC       30
LGERO       29
LGEEB       13
LGEAG        8
LGERA        8
LGECZ        7
LGELA        2
LGEUR        1
LGEIR        1
LGEBT        1
Name: count, dtype: int64

customer_country는 train과 test모두에서 drop하기로 하자. response_corporate와 겹친다. 

In [132]:
df_train['ver_win_ratio_per_bu'].value_counts()

ver_win_ratio_per_bu
0.064566    2460
0.049840    2028
0.020121    1365
0.048630    1356
0.026846    1278
0.064070    1075
0.071345     984
0.011583     788
0.022634     713
0.060924     540
0.079412     421
0.036913     403
0.035484     385
0.031579     271
0.053571     216
0.051471     210
0.028777     200
0.022727     173
0.128571     156
0.131148     148
0.227273      62
0.285714      43
0.034483      29
Name: count, dtype: int64

### 레이블 인코딩

In [5]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [6]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [7]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

### 2-2. 학습, 검증 데이터 분리

In [8]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모델 정의 

In [9]:
model = DecisionTreeClassifier()

### 모델 학습

In [10]:
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

### 모델 성능 보기

In [11]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [12]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  732   215]
 [  260 10653]]

정확도: 0.9599
정밀도: 0.7379
재현율: 0.7730
F1: 0.7550


## 4. 제출하기

### 테스트 데이터 예측

In [13]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [14]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

1173

### 제출 파일 작성

In [15]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_240214.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**